In [1]:
from common import *

cursor = connect()

# INSERT Trigger

创建触发器函数
```sql
CREATE OR REPLACE FUNCTION trigger_function()
   RETURNS TRIGGER
   LANGUAGE PLPGSQL
AS
$$
BEGIN
   -- trigger logic
   -- ...
   RETURN NEW;
END;
$$
```

In [14]:
sql = """
CREATE TABLE inventory_t(
    product_id INT PRIMARY KEY,
    quantity INT NOT NULL DEFAULT 0
);

CREATE TABLE inventory_stat(
    total_qty INT
);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a992194710>

In [15]:
sql = """
CREATE OR REPLACE FUNCTION update_total_qty()
RETURNS TRIGGER
LANGUAGE PLPGSQL
AS
$$
DECLARE
   p_row_count INT;
BEGIN
   SELECT COUNT(*) FROM inventory_stat
   INTO p_row_count;
   
   IF p_row_count > 0 THEN
      UPDATE inventory_stat 
      SET total_qty = total_qty + NEW.quantity;
   ELSE
      INSERT INTO inventory_stat(total_qty)
      VALUES(new.quantity);
   END IF;
   RETURN NEW;
END;
$$;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a992194710>

# BEFORE INSERT

语法
```
```sql
CREATE TRIGGER trigger_name
BEFORE INSERT
ON table_name
FOR EACH {ROW | STATEMENT}
EXECUTE FUNCTION trigger_function();
```
```

In [16]:
sql = """
CREATE TRIGGER inventory_before_insert
BEFORE INSERT
ON inventory_t
FOR EACH ROW
EXECUTE FUNCTION update_total_qty();

INSERT INTO inventory_t(product_id, quantity)
VALUES(1, 100)
RETURNING *;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a992194710>

In [17]:
sql = """
SELECT * FROM inventory_stat;
"""
run_sql(cursor, sql)

   total_qty
0        100


In [18]:
sql = """
INSERT INTO inventory_t(product_id, quantity)
VALUES(2, 200)
RETURNING *;
"""
cursor.execute(sql)

<psycopg.Cursor [TUPLES_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x1a992194710>

In [19]:
sql = """
SELECT * FROM inventory_stat;
"""
run_sql(cursor, sql)

   total_qty
0        300


# AFTER INSERT

语法
```
CREATE TRIGGER trigger_name
AFTER INSERT
ON table_name
FOR EACH {ROW | STATEMENT}
EXECUTE FUNCTION trigger_function();
```

In [2]:
sql = """
CREATE TABLE members (
    id SERIAL PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    email VARCHAR(100) UNIQUE
);

CREATE TABLE memberships (
    id SERIAL PRIMARY KEY,
    member_id INT NOT NULL REFERENCES members(id),
    membership_type VARCHAR(50) NOT NULL DEFAULT 'free'
);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x18843ca9550>

In [3]:
sql = """
CREATE OR REPLACE FUNCTION create_membership_after_insert()
RETURNS TRIGGER AS $$
BEGIN
    INSERT INTO memberships (member_id)
    VALUES (NEW.id);
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;

CREATE TRIGGER after_insert_member_trigger
AFTER INSERT ON members
FOR EACH ROW
EXECUTE FUNCTION create_membership_after_insert();
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost user=postgres database=dvdrental) at 0x18843ca9550>

In [4]:
sql = """
INSERT INTO members(name, email)
VALUES('John Doe', 'john.doe@gmail.com')
RETURNING *;
"""
run_sql(cursor, sql)

   id      name               email
0   1  John Doe  john.doe@gmail.com


In [5]:
sql = """
SELECT * FROM memberships;
"""
run_sql(cursor, sql)

   id  member_id membership_type
0   1          1            free
